In [27]:
from langchain_community.llms import Ollama
import chromadb

CHROMA_DATA_PATH = "../chroma_data/"
collection_db="article_comments"

chroma_client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

mistral = Ollama(model="mistral")
llama3 = Ollama(model="llama3:8b")
gemma = Ollama(model="gemma")

def ask_question_local(question:str, llm: Ollama() = llama3, 
                       collection: chromadb.PersistentClient() = collection_db, n_docs:int = 50, 
                       filters: dict ={}) -> str:
    
    # Find close documents in chromadb
    collection = chroma_client.get_collection(collection)
    results = collection.query(
       query_texts=[question],
       n_results=n_docs,
       where=filters
    )

    # Collect the results in a context
    context = "\n".join([r for r in results['documents'][0]])
    
    prompt = f"""
    Answer the following question: {question}.  
    Refer only to the following information when answering: {context}.
    Provide at least six paragraphs in summary if it is possible given the information provided.
    Begin your answer with 'Based on the responses from selected NY Times readers', and try to give a sense of majority and minority opinions on the topic, but only if there is an identifiable majority opinion.
    If there is not enough information provided to give a summarized opinion, indicate that this is the case.
    """
    
    print(llm.invoke(prompt))

In [28]:
ask_question_local("What do readers think about global warming?", llm = llama3)

Based on the responses from selected NY Times readers, it appears that the majority of readers are strongly supportive of the notion that global warming is real and man-made. Many readers express frustration with those who deny or downplay the issue, citing scientific consensus and evidence to support their claims.

Some readers, however, take a more nuanced approach, acknowledging the existence of global warming but questioning the degree to which human activity is responsible for the phenomenon. These readers often point to natural climate cycles and the complexity of climate systems as reasons why they are skeptical of the idea that human activity is solely or primarily responsible for global warming.

A smaller number of readers seem to be outright dismissive of the concept of global warming altogether, viewing it as a hoax or an overblown issue. These readers often express support for fossil fuels and criticize efforts to regulate carbon emissions, seeing them as an economic threa